In [5]:
import pandas as pd
import sqlite3
import asyncio
from ipyleaflet import Map, Marker, MarkerCluster, Popup, basemaps, AwesomeIcon
from ipywidgets import HTML, HBox, VBox, ToggleButton, IntText, Label
from datetime import datetime, timezone

connection = sqlite3.connect('cta_trains.db')

In [7]:
# --- 1) View builder (reuse your function) -----------------------------------
def ensure_view():
    """
    Creates and rebuilds the combined view 'trains_all' to include any newly created route tables.
    """
    cur = connection.cursor()
    all_lines = ["red", "blue", "brn", "g", "org", "p", "pink", "y"]
    lines_clean = { #Renaming lines to have full names
        "red": "Red", "blue": "Blue", "brn": "Brown", "g": "Green",
        "org": "Orange", "p": "Purple", "pink": "Pink", "y": "Yellow"
    }

    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    existing_tables = {row[0] for row in cur.fetchall()}
    valid_lines = [l for l in all_lines if l in existing_tables]
    if not valid_lines:
        return False  # nothing to build yet

    cur.execute("DROP VIEW IF EXISTS trains_all;")

    unions = "\nUNION ALL\n".join( #Appends all the data for all lines by iterating over each line table and add line_clean column
        [
            f"""SELECT
                    '{l}' AS line,
                    '{lines_clean[l]}' AS line_clean,
                    *
               FROM {l}"""
            for l in valid_lines
        ]
    )
    sql = f"CREATE VIEW trains_all AS\n{unions};"
    cur.executescript(sql)
    connection.commit()
    return True

# --- 2) One-time map + widgets ------------------------------------------------
line_colors = { #Specifying color for each train icon
    "Red": "red",
    "Blue": "blue",
    "Brown": "cadetblue",  # approximate
    "Green": "green",
    "Orange": "orange",
    "Purple": "purple",
    "Pink": "pink",
    "Yellow": "lightgray",
}

m = Map(basemap=basemaps.CartoDB.Positron, center=(41.8781, -87.6298), zoom=11)

status_lbl = Label(value="Starting…")
last_updated_html = HTML(value="")
play_toggle = ToggleButton(value=True, description="⏸ Pause / ▶️ Play", tooltip="Pause/resume auto-refresh")
interval_s = IntText(value=30, description="Interval (s):", tooltip="Seconds between refreshes")
ui = VBox([HBox([play_toggle, interval_s, status_lbl]), last_updated_html, m])
display(ui)

cluster_layer = None  # will hold the current MarkerCluster

# --- 3) Helpers ---------------------------------------------------------------
def query_latest_df():
    # Ensure the view exists (safe if rebuilt each cycle)
    ensure_view()

    # Pull the latest timestamp across the stacked view
    sql = """
        WITH max_ts AS (
            SELECT MAX(ts_utc) AS max_ts
            FROM trains_all
        )
        SELECT t.*
        FROM trains_all t
        JOIN max_ts ON t.ts_utc = max_ts.max_ts
    """
    df = pd.read_sql_query(sql, connection)

    # Normalize/guard
    if "ts_utc" in df.columns and not df.empty:
        # return both df and the max timestamp as a python datetime (UTC)
        ts = pd.to_datetime(df["ts_utc"].iloc[0], utc=True, errors="coerce")
    else:
        ts = pd.NaT
    return df, ts

def render_markers(df):
    global cluster_layer

    # Remove old cluster (keep base layer 0)
    if cluster_layer is not None:
        try:
            m.remove_layer(cluster_layer)
        except Exception:
            pass

    markers = []
    for _, row in df.iterrows():
        # Skip bad coords
        if pd.isna(row.get("lat")) or pd.isna(row.get("lon")):
            continue

        color = line_colors.get(str(row.get("line_clean")), "gray")
        icon = AwesomeIcon(name="train", marker_color=color, icon_color="white")

        rn = row.get("rn", "—")
        dest = row.get("next_station", "—")
        ts = row.get("ts_utc", "—")

        popup = HTML()
        popup.value = (
            f"<b>{row.get('line_clean','?')} Line</b><br>"
            f"Train #{rn}<br>"
            f"Headed to {dest}<br>"
            f"{ts} UTC"
        )

        markers.append(Marker(location=(float(row["lat"]), float(row["lon"])), icon=icon, popup=popup))

    cluster_layer = MarkerCluster(markers=markers)
    m.add_layer(cluster_layer)

# --- 4) Auto-refresh loop with pause detection --------------------------------
refresh_task = None

async def refresh_loop():
    """
    - Re-query latest rows every N seconds
    - Update the marker cluster in place
    - Auto-pause if the latest ts_utc hasn't advanced after 3 intervals
    """
    unchanged_count = 0
    last_seen_ts = None

    while True:
        try:
            if not play_toggle.value:
                status_lbl.value = "⏸ paused by user"
                await asyncio.sleep(1)
                continue

            df, max_ts = query_latest_df()
            now_utc = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")

            if df.empty or pd.isna(max_ts):
                unchanged_count += 1
                status_lbl.value = f"⚠️ no data yet (checks without change: {unchanged_count})"
            else:
                # Compare to last seen
                if last_seen_ts is None or (pd.notna(max_ts) and max_ts != last_seen_ts):
                    # New data -> render + reset unchanged counter
                    render_markers(df)
                    last_seen_ts = max_ts
                    unchanged_count = 0
                    status_lbl.value = "✅ updated"
                    last_updated_html.value = f"<i>Last map update:</i> {now_utc} • <i>Latest data ts:</i> {max_ts}"
                else:
                    unchanged_count += 1
                    status_lbl.value = f"⏳ waiting for new data (checks without change: {unchanged_count})"

            # Auto-pause if no change for 3 consecutive checks (~90s with 30s interval)
            if unchanged_count >= 3:
                play_toggle.value = False
                status_lbl.value = f"⏸ auto-paused (no new data for ~{unchanged_count * max(1, interval_s.value)}s)"
                # Keep markers as-is; user can press Play to resume

            await asyncio.sleep(max(1, int(interval_s.value)))
        except Exception as e:
            status_lbl.value = f"❌ error: {e}"
            # Back off a bit on errors
            await asyncio.sleep(max(5, int(interval_s.value)))

# Start/stop the loop when the toggle changes
def on_toggle_change(change):
    global refresh_task
    if change["name"] == "value":
        if change["new"] and (refresh_task is None or refresh_task.done()):
            status_lbl.value = "▶️ resuming…"
            refresh_task = asyncio.create_task(refresh_loop())
        elif not change["new"]:
            status_lbl.value = "⏸ paused"
            # We don't cancel the task immediately; loop respects the toggle and idles

play_toggle.observe(on_toggle_change)

# Kick off once
if refresh_task is None or refresh_task.done():
    refresh_task = asyncio.create_task(refresh_loop())